# Avec La librairy geopy

In [1]:
from geopy.geocoders import Nominatim

In [58]:
geolocator = Nominatim(user_agent="VocalWeatherMelody")

In [60]:
location = geolocator.geocode("Tours nord")
print(f"latitude: {location.latitude}, longitude: {location.longitude}")

latitude: 47.4080507, longitude: 0.7216688


# Avec l'api geocoding d'open meteo

In [13]:
import requests
import json

In [39]:
def simple_get_request(location):
  try:
    response = requests.get(f"https://geocoding-api.open-meteo.com/v1/search?name={location}&count=1&language=fr&format=json")
    response.raise_for_status()
    return response
  except requests.exceptions.RequestException as e:
    print(f"Error during GET request: {e}")
    return None

In [57]:
response = simple_get_request("Tours")

In [41]:
if response:
    print(response.status_code)
    informations = json.loads(response.text)
    latitude = informations["results"][0]["latitude"]
    longitude = informations["results"][0]["longitude"]
    print(f"latitude: {latitude}, longitude: {longitude}")

200
latitude: 47.39484, longitude: 0.70398


# Avec l'api de adresse data gouv

In [31]:
import requests
import json

In [49]:
def simple_get_request(location):
  try:
    response = requests.get(f"https://api-adresse.data.gouv.fr/search/?q={location}&limit=1")
    response.raise_for_status()
    return response
  except requests.exceptions.RequestException as e:
    print(f"Error during GET request: {e}")
    return None

In [56]:
response = simple_get_request("Tours")

In [55]:
if response:
    print(response.status_code)
    informations = json.loads(response.text)
    latitude = informations["features"][0]["geometry"]["coordinates"][1]
    longitude = informations["features"][0]["geometry"]["coordinates"][0]
    print(f"latitude: {latitude}, longitude: {longitude}")

200
latitude: 47.395476, longitude: 0.694273


Ce dernier a l'air d'avoir un temps de traitement légèrement plus rapide et n'a pas de limite quotidienne.

# Benchmark des api de geolocalisations

In [74]:

geolocator = Nominatim(user_agent="VocalWeatherMelody")
start_time = time.time()
location = geolocator.geocode("Tours nord")
end_time = time.time()
geopy_time = end_time - start_time
print(f"Geopy: latitude: {location.latitude}, longitude: {location.longitude}, temps: {geopy_time:.4f} secondes")

def simple_get_request_openmeteo(location):
  try:
    start_time = time.time()
    response = requests.get(f"https://geocoding-api.open-meteo.com/v1/search?name={location}&count=1&language=fr&format=json")
    response.raise_for_status()
    end_time = time.time()
    return response, end_time - start_time
  except requests.exceptions.RequestException as e:
    print(f"Error during GET request (Open-Meteo): {e}")
    return None, None

response, openmeteo_time = simple_get_request_openmeteo("Tours")
if response:
    informations = json.loads(response.text)
    latitude = informations["results"][0]["latitude"]
    longitude = informations["results"][0]["longitude"]
    print(f"Open-Meteo: latitude: {latitude}, longitude: {longitude}, temps: {openmeteo_time:.4f} secondes")

def simple_get_request_datagouv(location):
  try:
    start_time = time.time()
    response = requests.get(f"https://api-adresse.data.gouv.fr/search/?q={location}&limit=1")
    response.raise_for_status()
    end_time = time.time()
    return response, end_time - start_time
  except requests.exceptions.RequestException as e:
    print(f"Error during GET request (Data.gouv): {e}")
    return None, None

response, datagouv_time = simple_get_request_datagouv("Tours")
if response:
    informations = json.loads(response.text)
    latitude = informations["features"][0]["geometry"]["coordinates"][1]
    longitude = informations["features"][0]["geometry"]["coordinates"][0]
    print(f"Data.gouv: latitude: {latitude}, longitude: {longitude}, temps: {datagouv_time:.4f} secondes")

print("\nComparaison des temps de traitement:")
print(f"Geopy: {geopy_time:.4f} secondes")
print(f"Open-Meteo: {openmeteo_time:.4f} secondes")
print(f"Data.gouv: {datagouv_time:.4f} secondes")



Geopy: latitude: 47.4080507, longitude: 0.7216688, temps: 0.0883 secondes
Open-Meteo: latitude: 47.39484, longitude: 0.70398, temps: 0.0763 secondes
Data.gouv: latitude: 47.395476, longitude: 0.694273, temps: 0.0621 secondes

Comparaison des temps de traitement:
Geopy: 0.0883 secondes
Open-Meteo: 0.0763 secondes
Data.gouv: 0.0621 secondes
